## Prerequisites

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Observation of data

Let's see how our data look like

In [2]:
train_data = pd.read_csv('data/train.csv', header=None)

In [3]:
train_data

,0,1,2,3,4
0,0,0.012495,0.011126,0.003252,0.006625
1,1,0.011439,0.002691,0.001206,0.006947
2,2,0.000632,0.007277,0.004049,0.000074
3,3,0.017828,0.028210,0.007758,0.007382
4,4,0.021115,0.019642,0.009238,0.011499
...,...,...,...,...,...
741,741,0.001938,0.008833,0.003927,0.005106
742,742,0.005003,0.018943,0.003057,0.001988
743,743,0.007683,0.001958,0.007002,0.006467
744,744,0.003396,0.001280,0.007621,0.001680


In [4]:
X = np.array(train_data.drop(labels=0, axis=1))
X.shape

(746, 4)

In [5]:
data_dim = X.shape[1]

### Definition of GAN models

In [6]:
import torch
import torch.nn as nn

In [7]:
if torch.cuda.is_available():
    print('You use GPU !')
    device = torch.device('cuda')
else:
    print('You use CPU !')
    device = torch.device('cpu')

You use GPU !


In [8]:
# Generative model
def make_generator(noise_dim):
    return nn.Sequential(
        nn.Linear(in_features=noise_dim, out_features=256),
        nn.BatchNorm1d(256),
        nn.LeakyReLU(),
        
        nn.Linear(in_features=256, out_features=64),
        nn.BatchNorm1d(64),
        nn.LeakyReLU(),
        
        nn.Linear(in_features=64, out_features=data_dim),
        nn.ReLU(),
    )


# Discriminative model
def make_discriminator():
    return nn.Sequential(
        nn.Linear(in_features=data_dim, out_features=256),
        nn.BatchNorm1d(256),
        nn.Sigmoid(),
        
        nn.Linear(in_features=256, out_features=64),
        nn.BatchNorm1d(64),
        nn.Sigmoid(),
        
        nn.Linear(in_features=64, out_features=2)
    )

In [9]:
noise_dim = 10
generator = make_generator(noise_dim=noise_dim).to(device)
discriminator = make_discriminator().to(device)

In [10]:
generator

Sequential(
  (0): Linear(in_features=10, out_features=256, bias=True)
  (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): LeakyReLU(negative_slope=0.01)
  (3): Linear(in_features=256, out_features=64, bias=True)
  (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=64, out_features=4, bias=True)
  (7): ReLU()
)

In [11]:
discriminator

Sequential(
  (0): Linear(in_features=4, out_features=256, bias=True)
  (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): Sigmoid()
  (3): Linear(in_features=256, out_features=64, bias=True)
  (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): Sigmoid()
  (6): Linear(in_features=64, out_features=2, bias=True)
)

## Training

Split dataset into training data and evaluation data

In [12]:
X = torch.tensor(np.array(train_data.drop(labels=0, axis=1)), dtype=torch.float)
mu = X.mean(dim=0)
theta = torch.sqrt(((X - mu)**2).mean(dim=0))

In [13]:
def norm(x, mu, theta):
    return (x - mu)/theta

def denorm(x, mu, theta):
    return x * theta + mu

In [14]:
X = norm(X, mu, theta)

In [15]:
eval_rate = 0.2
eval_index = int(X.shape[0] * (1 - eval_rate))
X_train = X[0: eval_index]
X_eval = X[eval_index:]

Create dataloaders

In [16]:
batch_size = 64

X_train_loader = torch.utils.data.DataLoader(X_train, batch_size, shuffle=True)
X_eval_loader = torch.utils.data.DataLoader(X_eval, batch_size, shuffle=True)

We choose Adam Optimizer ($\beta_1 = 0.9$, $\beta_2 = 0.999$)

In [17]:
num_epochs = 500
lr = 0.001
gen_optimizer = torch.optim.Adam(generator.parameters(), lr=lr)
disc_optimizer = torch.optim.Adam(discriminator.parameters(), lr=lr)

criterion = torch.nn.CrossEntropyLoss()

In [18]:
for i in range(num_epochs):
    print(f'Epoch {i+1}\t', end='')
    disc_losses = []
    gen_losses = []
    for true_data in X_train_loader:
        #### DISCRIMINATOR OPTMIZATION ####
        n_data = true_data.shape[0]
        
        discriminator.train() # Unfreeze parameters of the discriminator
        generator.eval() # Freeze parameters of the generator
        
        disc_optimizer.zero_grad()
        
        true_x = true_data.to(device) # true data from the training dataset
        noise = torch.randn(n_data, noise_dim).to(device)
        fake_x = generator(noise).detach() # fake data from the noise distribution ~ N(0, 1)

        x = torch.cat([true_x, fake_x]) # Gather true and fake data
        
        
        true_y = torch.ones((n_data,), dtype=torch.long).to(device) # target 1 for true data
        fake_y = torch.zeros((n_data,), dtype=torch.long).to(device) # target 0 for fake data
        
        y = torch.cat([true_y, fake_y]) # Gather true and fake targets
        
        
        
        output = discriminator(x)
        
        disc_loss = criterion(output, y) # Penalize Discriminator for failing to distinguish fake data from true data
        disc_losses.append(disc_loss.item())
        
        disc_loss.backward()
        
        
        disc_optimizer.step() # Optimize Discriminator
        
        
        
        ### GENERATOR OPTIMIZATION ###
        n_data = true_data.shape[0] * 4
    
        discriminator.eval() # Freeze parameters of the discriminator
        generator.train() # Unfreeze parameters of the generator

        gen_optimizer.zero_grad()
        
        
        noise = torch.randn(n_data, noise_dim).to(device)
        fake_x = generator(noise) # fake data from the noise distribution ~ N(0, 1)
        
        true_y = torch.ones((n_data,), dtype=torch.long).to(device) # target 1 for true data
        
        output = discriminator(fake_x)
        
        gen_loss = criterion(output, true_y) # Penalize Generator for failing to fool the discriminator
        gen_losses.append(gen_loss.item())
        
        gen_loss.backward()
        

        gen_optimizer.step() # Optimize Generator  
    
    
    disc_loss = np.mean(disc_losses)
    gen_loss = np.mean(gen_losses)
    
    print(f'Disc. loss: {disc_loss}\t', end='')
    print(f'Gen. loss: {gen_loss}')
    

Epoch 1	Disc. loss: 0.6615031003952027	Gen. loss: 0.784655487537384
Epoch 2	Disc. loss: 0.6444855034351349	Gen. loss: 0.802925193309784
Epoch 3	Disc. loss: 0.6654925525188446	Gen. loss: 0.752035665512085
Epoch 4	Disc. loss: 0.6570060312747955	Gen. loss: 0.7522339642047882
Epoch 5	Disc. loss: 0.6433203995227814	Gen. loss: 0.7690511226654053
Epoch 6	Disc. loss: 0.6386339664459229	Gen. loss: 0.8363366782665252
Epoch 7	Disc. loss: 0.6245865225791931	Gen. loss: 0.8306571424007416
Epoch 8	Disc. loss: 0.6037003517150878	Gen. loss: 0.9170199751853942
Epoch 9	Disc. loss: 0.5574246346950531	Gen. loss: 1.1058687150478363
Epoch 10	Disc. loss: 0.5001190006732941	Gen. loss: 1.3711613416671753
Epoch 11	Disc. loss: 0.43972674012184143	Gen. loss: 1.7944088697433471
Epoch 12	Disc. loss: 0.42942791879177095	Gen. loss: 2.2055188179016114
Epoch 13	Disc. loss: 0.35395767986774446	Gen. loss: 2.5256167888641357
Epoch 14	Disc. loss: 0.3154776573181152	Gen. loss: 2.7984646558761597
Epoch 15	Disc. loss: 0.268516

Epoch 120	Disc. loss: 0.3715049773454666	Gen. loss: 1.3776726365089416
Epoch 121	Disc. loss: 0.3663812935352325	Gen. loss: 1.3831565260887146
Epoch 122	Disc. loss: 0.38091476261615753	Gen. loss: 1.3772352814674378
Epoch 123	Disc. loss: 0.36439929008483884	Gen. loss: 1.5318590879440308
Epoch 124	Disc. loss: 0.3757751166820526	Gen. loss: 1.427037286758423
Epoch 125	Disc. loss: 0.39669119417667387	Gen. loss: 1.56555677652359
Epoch 126	Disc. loss: 0.369711372256279	Gen. loss: 1.5496949076652526
Epoch 127	Disc. loss: 0.35482153594493865	Gen. loss: 1.5303675651550293
Epoch 128	Disc. loss: 0.37688611447811127	Gen. loss: 1.6037490487098693
Epoch 129	Disc. loss: 0.37465394139289854	Gen. loss: 1.6110775232315064
Epoch 130	Disc. loss: 0.3512293934822083	Gen. loss: 1.7059202075004578
Epoch 131	Disc. loss: 0.34711042046546936	Gen. loss: 1.4977964282035827
Epoch 132	Disc. loss: 0.3535922408103943	Gen. loss: 1.6490043640136718
Epoch 133	Disc. loss: 0.34471082091331484	Gen. loss: 1.6365487098693847
Ep

Epoch 236	Disc. loss: 0.3401243597269058	Gen. loss: 1.6558172583580018
Epoch 237	Disc. loss: 0.3138796463608742	Gen. loss: 1.5823472380638122
Epoch 238	Disc. loss: 0.3047815814614296	Gen. loss: 1.515380871295929
Epoch 239	Disc. loss: 0.31638005673885344	Gen. loss: 1.6943799376487731
Epoch 240	Disc. loss: 0.31894259750843046	Gen. loss: 1.5707138419151305
Epoch 241	Disc. loss: 0.30855591893196105	Gen. loss: 1.8862343549728393
Epoch 242	Disc. loss: 0.3134388029575348	Gen. loss: 1.635286033153534
Epoch 243	Disc. loss: 0.3170053392648697	Gen. loss: 1.628914201259613
Epoch 244	Disc. loss: 0.2981032535433769	Gen. loss: 1.8442535758018495
Epoch 245	Disc. loss: 0.29144154489040375	Gen. loss: 1.8961447715759276
Epoch 246	Disc. loss: 0.28581011295318604	Gen. loss: 1.9421762108802796
Epoch 247	Disc. loss: 0.283208429813385	Gen. loss: 1.9662811875343322
Epoch 248	Disc. loss: 0.31264127790927887	Gen. loss: 2.125841665267944
Epoch 249	Disc. loss: 0.32192196547985075	Gen. loss: 1.866879951953888
Epoch

Epoch 352	Disc. loss: 0.31066514253616334	Gen. loss: 2.1222018361091615
Epoch 353	Disc. loss: 0.29252957701683047	Gen. loss: 1.752414071559906
Epoch 354	Disc. loss: 0.2820185899734497	Gen. loss: 1.9546061992645263
Epoch 355	Disc. loss: 0.3075859323143959	Gen. loss: 1.9324840307235718
Epoch 356	Disc. loss: 0.2920322477817535	Gen. loss: 1.7749521613121033
Epoch 357	Disc. loss: 0.29260424226522447	Gen. loss: 1.9022231578826905
Epoch 358	Disc. loss: 0.2985881924629211	Gen. loss: 1.7934437274932862
Epoch 359	Disc. loss: 0.3033438980579376	Gen. loss: 1.7288247704505921
Epoch 360	Disc. loss: 0.2874980464577675	Gen. loss: 1.8387707948684693
Epoch 361	Disc. loss: 0.2874173760414124	Gen. loss: 1.8188298702239991
Epoch 362	Disc. loss: 0.2881524160504341	Gen. loss: 1.7882270097732544
Epoch 363	Disc. loss: 0.28658811151981356	Gen. loss: 1.9082687854766847
Epoch 364	Disc. loss: 0.2792623907327652	Gen. loss: 1.8794975876808167
Epoch 365	Disc. loss: 0.2867651268839836	Gen. loss: 2.086746597290039
Epoc

Epoch 469	Disc. loss: 0.2588779076933861	Gen. loss: 2.11241672039032
Epoch 470	Disc. loss: 0.25131491124629973	Gen. loss: 2.0913439750671388
Epoch 471	Disc. loss: 0.25567798018455506	Gen. loss: 2.055191922187805
Epoch 472	Disc. loss: 0.28386152982711793	Gen. loss: 2.057574462890625
Epoch 473	Disc. loss: 0.26811091154813765	Gen. loss: 1.786786103248596
Epoch 474	Disc. loss: 0.26578719168901443	Gen. loss: 2.0264625191688537
Epoch 475	Disc. loss: 0.2588771864771843	Gen. loss: 1.8848705410957336
Epoch 476	Disc. loss: 0.26912054866552354	Gen. loss: 2.1307661533355713
Epoch 477	Disc. loss: 0.26859532445669176	Gen. loss: 1.889698040485382
Epoch 478	Disc. loss: 0.2701322391629219	Gen. loss: 2.099729299545288
Epoch 479	Disc. loss: 0.263787205517292	Gen. loss: 1.8567912340164185
Epoch 480	Disc. loss: 0.27165959030389786	Gen. loss: 1.7822728276252746
Epoch 481	Disc. loss: 0.254658454656601	Gen. loss: 1.8584864377975463
Epoch 482	Disc. loss: 0.2693925142288208	Gen. loss: 2.0648998737335207
Epoch 4

In [19]:
discriminator.eval()
generator.eval()

Sequential(
  (0): Linear(in_features=10, out_features=256, bias=True)
  (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): LeakyReLU(negative_slope=0.01)
  (3): Linear(in_features=256, out_features=64, bias=True)
  (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=64, out_features=4, bias=True)
  (7): ReLU()
)

In [20]:
discriminator(generator(torch.randn(batch_size, noise_dim).to(device))).argmax(dim=1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')

In [21]:
discriminator(X_eval.to(device)).argmax(dim=1)

tensor([1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
        1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0,
        0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1,
        1, 1, 1, 1, 1, 1], device='cuda:0')

In [22]:
denorm(generator(torch.randn(batch_size, noise_dim).to(device)), mu.to(device), theta.to(device))

tensor([[0.0131, 0.0128, 0.0094, 0.0108],
        [0.0242, 0.0307, 0.0338, 0.0211],
        [0.0317, 0.0236, 0.0154, 0.0108],
        [0.0228, 0.0258, 0.0274, 0.0241],
        [0.0343, 0.0219, 0.0172, 0.0149],
        [0.0427, 0.0604, 0.0329, 0.0245],
        [0.0131, 0.0154, 0.0094, 0.0108],
        [0.0131, 0.0128, 0.0094, 0.0108],
        [0.0248, 0.0265, 0.0094, 0.0188],
        [0.0131, 0.0128, 0.0094, 0.0108],
        [0.0258, 0.0277, 0.0094, 0.0108],
        [0.0763, 0.0258, 0.0153, 0.0223],
        [0.0227, 0.0128, 0.0094, 0.0195],
        [0.0131, 0.0128, 0.0163, 0.0108],
        [0.0356, 0.0383, 0.0370, 0.0344],
        [0.0131, 0.0213, 0.0094, 0.0108],
        [0.0779, 0.0568, 0.0366, 0.0509],
        [0.0238, 0.0298, 0.0332, 0.0248],
        [0.0197, 0.0212, 0.0192, 0.0182],
        [0.0522, 0.0501, 0.0165, 0.0108],
        [0.0235, 0.0128, 0.0094, 0.0217],
        [0.0131, 0.0128, 0.0094, 0.0108],
        [0.0131, 0.0128, 0.0180, 0.0162],
        [0.0236, 0.0298, 0.0094, 0